In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/GreenStand/Pilot_Binary/Models/

/content/drive/MyDrive/GreenStand/Pilot_Binary/Models


In [ ]:
!pip install -r requirements.txt

In [46]:
!python3 binary_leaf_main.py --model deeplabv3plus_mobilenet --custom_data_path "../PV_Sub" --batch_size 16 --lr 0.12 --total_itrs 800 --loss_type BCE --val_interval 100  --separable_conv #--lr_policy step

Device: cuda
Dataset: custom, Train set: 140, Val set: 40
[!] Retrain
Epoch 2, Itrs 10/800, Loss=0.538440
Epoch 3, Itrs 20/800, Loss=0.316857
Epoch 4, Itrs 30/800, Loss=0.274341
Epoch 5, Itrs 40/800, Loss=0.264379
Epoch 7, Itrs 50/800, Loss=0.254805
Epoch 8, Itrs 60/800, Loss=0.245035
Epoch 9, Itrs 70/800, Loss=0.241418
Epoch 10, Itrs 80/800, Loss=0.239791
Epoch 12, Itrs 90/800, Loss=0.228301
Epoch 13, Itrs 100/800, Loss=0.220867
Model saved as checkpoints/latest_deeplabv3plus_mobilenet_custom_os16.pth
validation...
10it [00:00, 14.44it/s]
Validation Loss: 0.250765

Foreground Acc: 0.873871
Precision: 0.950607
Recall: 0.873871
F1 Score: 0.910625
IoU Foreground: 0.835915
IoU Background: 0.847086
Mean IoU: 0.841501

Model saved as checkpoints/best_deeplabv3plus_mobilenet_custom_os16.pth
Epoch 14, Itrs 110/800, Loss=0.231027
Epoch 15, Itrs 120/800, Loss=0.221101
Epoch 17, Itrs 130/800, Loss=0.219509
Epoch 18, Itrs 140/800, Loss=0.224347
Epoch 19, Itrs 150/800, Loss=0.223564
Epoch 20, Itrs